In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import hashlib

In [12]:
load_dotenv()
data_path1 = os.getenv("DATA_PATH1")

df1 = pd.read_csv(data_path1)

In [13]:
# 불필요한 컬럼 삭제
df1 = df1.drop(columns=['부채상환계수.1'], errors='ignore')
print("'부채상환계수.1' 컬럼 삭제 완료")

# 기준년월이 20210801인 데이터 개수 확인
count_20210801 = df1[df1['기준년월'] == 20210801].shape[0]
print(f"기준년월이 20210801인 행의 개수: {count_20210801}")

# 기준년월이 20210801인 행만 필터링
df_20210801 = df1[df1['기준년월'] == 20210801]

# 저장 경로 지정
save_path = r"C:\Users\user\rde-data\data.csv"

# 새 CSV 파일로 저장
df_20210801.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"저장 완료")


'부채상환계수.1' 컬럼 삭제 완료
기준년월이 20210801인 행의 개수: 50000
저장 완료


In [19]:
load_dotenv()
data_path = os.getenv("DATA")

df = pd.read_csv(data_path)

In [20]:
len(df)

50000

In [23]:
# 널값이 있는 컬럼과 널값 개수 조회
null_counts = df.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

print("🔍 널값이 있는 컬럼 및 개수")
display(null_counts)

🔍 널값이 있는 컬럼 및 개수


상장폐지일자          49959
상장일자            48268
재무비율_총자산순이익률    31850
재무비율_당기순이익율     31539
주소지시군구           4097
재무비율_유형자산증가율     3600
재무비율_유동자산증가율     3600
재무비율_매입채무회전율     3600
EBITDA증가율        1906
영업이익이자보상배율         24
이자보상배율             24
업종(중분류)             5
dtype: int64

In [ ]:
# df 또는 df1 중 사용하는 이름으로 맞춰 주세요 (여기선 df를 기준으로 작성)
# df = df1  # <- 만약 현재 변수명이 df1이라면 주석 해제

def safe_div(num, den):
    out = np.where((pd.notna(num)) & (pd.notna(den)) & (den != 0), num / den, np.nan)
    return pd.Series(out, index=num.index)

def clean_num(s: pd.Series):
    """무한대/비수치 처리"""
    s = pd.to_numeric(s, errors='coerce')
    s = s.replace([np.inf, -np.inf], np.nan)
    return s

def fill_metric(colname, calc_series):
    """기존 값이 NaN인 곳만 계산값으로 채우기 (인덱스 정렬 기반)"""
    if colname not in df.columns:
        df[colname] = np.nan
    calc_series = clean_num(calc_series).reindex(df.index)
    df[colname] = df[colname].fillna(calc_series)

# ---------------- 계산 & 채움 ----------------

# 1) 재무비율_총자산순이익률 = (당기순이익 / 자산총계) × 100
if {'당기순이익','자산총계'}.issubset(df.columns):
    calc = safe_div(df['당기순이익'], df['자산총계']) * 100
    fill_metric('재무비율_총자산순이익률', calc)

# 2) 재무비율_당기순이익율 = (당기순이익 / 매출액) × 100
if {'당기순이익','매출액'}.issubset(df.columns):
    calc = safe_div(df['당기순이익'], df['매출액']) * 100
    fill_metric('재무비율_당기순이익율', calc)

# 3) 재무비율_매입채무회전율 = 매출원가 / 매입채무
if {'매출원가','매입채무'}.issubset(df.columns):
    calc = safe_div(df['매출원가'], df['매입채무'])
    fill_metric('재무비율_매입채무회전율', calc)

# 4) EBITDA증가율 = (EBITDA - EBITDA(전기)) / EBITDA(전기) × 100
ebitda_prev = None
if 'EBITDA(전기)' in df.columns:
    ebitda_prev = df['EBITDA(전기)']
elif {'가명식별자','기준년월'}.issubset(df.columns) and 'EBITDA' in df.columns:
    # 기업별 전기 생성
    df_sorted = df.sort_values(['가명식별자','기준년월'])
    ebitda_prev = df_sorted.groupby('가명식별자', group_keys=False)['EBITDA'].shift(1)
    # 원래 인덱스로 재정렬해 맞춰줌
    ebitda_prev = ebitda_prev.reindex(df_sorted.index)
    ebitda_prev = ebitda_prev.reindex(df.index)  # 최종적으로 df 인덱스에 맞춤

if ('EBITDA' in df.columns) and (ebitda_prev is not None):
    calc = safe_div(df['EBITDA'] - ebitda_prev, ebitda_prev) * 100
    fill_metric('EBITDA증가율', calc)

# 5) 영업이익이자보상배율 = 영업손익 / 금융비용
if {'영업손익','금융비용'}.issubset(df.columns):
    calc = safe_div(df['영업손익'], df['금융비용'])
    fill_metric('영업이익이자보상배율', calc)

# 6) 이자보상배율 = EBIT / 금융비용
if {'EBIT','금융비용'}.issubset(df.columns):
    calc = safe_div(df['EBIT'], df['금융비용'])
    fill_metric('이자보상배율', calc)

# ---------------- 결과 요약 ----------------
targets = [
    '재무비율_총자산순이익률',
    '재무비율_당기순이익율',
    '재무비율_매입채무회전율',
    'EBITDA증가율',
    '영업이익이자보상배율',
    '이자보상배율'
]
summary = (
    pd.DataFrame({
        'NaN개수': [df[c].isna().sum() if c in df.columns else '미생성' for c in targets],
        '비고': ['' for _ in targets],
    }, index=targets)
)
print("✅ 계산식 기반 채우기 완료! 지표별 NaN 개수:")
display(summary)

load_dotenv()
save_path = os.getenv("DATA")
df.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"CSV 저장 완료: '{save_path}' 에 최신 계산 결과가 덮어써졌습니다.")


✅ 계산식 기반 채우기 완료! 지표별 NaN 개수:


,NaN개수,비고
재무비율_총자산순이익률,0,
재무비율_당기순이익율,0,
재무비율_매입채무회전율,10518,
EBITDA증가율,1906,
영업이익이자보상배율,2,
이자보상배율,2,


💾 CSV 저장 완료: 'C:\Users\user\rde-data\data.csv' 에 최신 계산 결과가 덮어써졌습니다.


In [ ]:
# 널값이 있는 컬럼과 널값 개수 조회
null_counts = df.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

print("널값이 있는 컬럼 및 개수")
display(null_counts)

🔍 널값이 있는 컬럼 및 개수


상장폐지일자          49959
상장일자            48268
재무비율_매입채무회전율    10518
주소지시군구           4097
재무비율_유형자산증가율     3600
재무비율_유동자산증가율     3600
EBITDA증가율        1906
업종(중분류)             5
이자보상배율              2
영업이익이자보상배율          2
dtype: int64

In [38]:
# 중복 데이터 확인
duplicate_rows = df[df.duplicated(keep=False)]

print(f"🔍 중복된 행 개수: {duplicate_rows.shape[0]}")

🔍 중복된 행 개수: 0


In [ ]:
# 타겟 데이터 확인
value_counts = df['모형개발용Performance(향후1년내부도여부)'].value_counts(dropna=False)
display(value_counts)

모형개발용Performance(향후1년내부도여부)
0    49242
1      758
Name: count, dtype: int64